In [2]:

from transformers import pipeline, M2M100ForConditionalGeneration, M2M100Tokenizer
from langdetect import detect


In [3]:
# Summarization model
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)

# Translation model
translator_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
translator_model = M2M100ForConditionalGeneration.from_pretrained(
    "facebook/m2m100_418M"
)


Device set to use cpu


In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "en"


In [5]:
def translate(text, src, tgt):
    translator_tokenizer.src_lang = src
    encoded = translator_tokenizer(text, return_tensors="pt", truncation=True)

    generated = translator_model.generate(
        **encoded,
        forced_bos_token_id=translator_tokenizer.get_lang_id(tgt)
    )

    return translator_tokenizer.decode(
        generated[0],
        skip_special_tokens=True
    )


In [6]:
def summarize_text(text):
    return summarizer(
        text,
        max_length=130,
        min_length=40,
        do_sample=False
    )[0]["summary_text"]


In [7]:
def process_article(text):
    lang = detect_language(text)

    if lang != "en":
        text = translate(text, lang, "en")

    summary = summarize_text(text)

    return {
        "detected_language": lang,
        "english_summary": summary
    }


In [8]:
article = "Daryl Mitchell stood tall with a stroke-filled half-century to help New Zealand post 300 for 8 in the first One-Day International against India in Vadodara on Sunday (January 11, 2026). Mitchell, ranked world No. 3 behind India superstars Virat Kohli and Rohit Sharma, lived up to expectations with a gritty knock which gave New Zealand a much-needed impetus amid middle order collapse. Mitchell clobbered five fours and three sixes to make 84 off 71 balls after New Zealand middle order failed to build on an ideal start given by Devon Conway (56 off 67 balls; 6 fours, 1 six) and Henry Nicholls (62 off 69 balls; 8 fours). Openers Conway and Nicholls had put on 117 runs to defy India for the first 21 overs, but Harshit Rana’s (2/65) second spell of 2-0-13-2 turned the tables and the home side chipped away with regular strikes. New Zealand skidded from 117 for no loss in the 22nd over to 198 for five in the 38th. But Mitchell forged vital stands to come to their rescue, while debutant Kristian Clarke struck three late fours to make 17-ball 24 not out. In the 22nd over, Rana took pace off the ball to break the century partnership between Conway and Nicholls. An innocuous off-cutter, bowled away from the batter saw Nicholls reaching out for it, only to get an edge that carried to the wicketkeeper for India’s first breakthrough. In the 24th over, Rana mixed up the slower ones with a few in excess of 140 kmph to keep the Kiwis guessing. On the final delivery of the over, he had Conway’s inside edge crashing into the wickets. But India and Rana would not have had to wait for a breakthrough had Kuldeep Yadav (1/52) hung on to a regulation catch at third man in the fifth over, when Nicholls was on five. Siraj sent down a slower one which had Will Young feathering one behind the wickets, while Prasidh Krishna (2/60) cleaned up Mitchell Hay (12) in the 37th. Under the weather for the last few days, Glenn Phillips appeared a tad sluggish in his brief stay which was ended by Kuldeep, and a brilliant direct hit by Shreyas Iyer from long-on caught Kiwi skipper Michael Bracewell short of his crease. Earlier, Conway began with a drive down the wicket for his first boundary while Nicholls took his time to get going, and used the sweep shot well at times to find the odd boundary. The duo grew in confidence to bring out the sweep and reverse sweeps with perfection while rotating the strike well to put on the century stand."
result = process_article(article)
result


{'detected_language': 'en',
 'english_summary': 'Daryl Mitchell hits 84 off 71 balls to help New Zealand post 300 for 8. Mitchell is ranked world No. 3 behind Virat Kohli and Rohit Sharma. New Zealand skidded from 117 for no loss in the 22nd over to 198 for five in the 38th.'}